In [1]:
import numpy as np
import random
import cv2
import os
from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import json
import tensorflow as tf
#import tensorflow_federated as tff
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
#tf.config.set_visible_devices([], 'GPU')

In [2]:
DATA_PATH = r"C:\Users\sreed\VSCODE_CoDING\Project_Phase_1\Speech_strand\Features\Federated_MFCC_features.json"

In [3]:
def load_data(data_path):

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])
    return X, y

In [4]:
def prepare_datasets(test_size):

    # load data
    X, Y = load_data(DATA_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size)
    #X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    #X_train = X_train[..., np.newaxis]
    #X_test = X_test[..., np.newaxis]
    
    print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

    return X_train, X_test, y_train, y_test

In [5]:
def create_clients(image_list, label_list, num_clients=2, initial='clients'):
    
    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(image_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))} 

In [6]:
X_train, X_test, y_train, y_test = prepare_datasets(0.2)
clients = create_clients(X_train, y_train, num_clients=2, initial='client')

(4480, 65, 13) (1120, 65, 13) (4480,) (1120,)


In [7]:
def batch_data(data_shard, bs=32):
    
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    #print(data,label)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    print(dataset)
    return dataset.shuffle(len(label)).batch(bs)

In [8]:
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

<TensorSliceDataset element_spec=(TensorSpec(shape=(65, 13), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>
<TensorSliceDataset element_spec=(TensorSpec(shape=(65, 13), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>


In [9]:
def global_LSTM(shape):
    model = Sequential()
    model.add(keras.layers.LSTM(128, input_shape=shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(7, activation='softmax'))
    return model

def global_BiLSTM(shape):
    model = keras.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(128,return_sequences=False),input_shape=shape))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(7, activation='softmax'))
    return model

def global_CNN(input_shape):

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model



def global_CNN_LSTM(input_shape):

    nb_filter = 50 # 50 output channels
    kernel_size_x = input_shape[0]//10 # 50 -> 5
    kernel_size_y = input_shape[1] # 48

    model = keras.Sequential()
    model.add(keras.layers.Conv2D(nb_filter, (kernel_size_x, kernel_size_y), input_shape=input_shape, activation='relu'))
    # model.add(MaxPool2D(pool_size=(2, 2))) by imposing kernel_size_y to #features
    model.add(keras.layers.Dropout(0.2))
    #     reshape(-1,data.shape[1]).shape
    model.add(keras.layers.Reshape(target_shape=(-1, nb_filter)))
    model.add(keras.layers.LSTM(64, return_sequences=False))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(2, activation='softmax'))
    print('Compile model...')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def global_lstm_gru(input_shape):
    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.GRU(64))

    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Dense(7, activation='softmax'))
    
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

    return model


def global_GRU(input_shape):

    # build network topology
    model = keras.Sequential()

    model.add(keras.layers.GRU(128, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.GRU(64))

          # dense layer
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

          # output layer
    model.add(keras.layers.Dense(7, activation='softmax'))

    return model

In [10]:
def local_LSTM(shape):
    model = Sequential()
    model.add(keras.layers.LSTM(128, input_shape=shape, return_sequences=True))
    model.add(keras.layers.LSTM(64))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Dense(7, activation='softmax'))
    return model

def local_BiLSTM(shape):
    model = keras.Sequential()
    model.add(keras.layers.Bidirectional(keras.layers.LSTM(128,return_sequences=False),input_shape=shape))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(7, activation='softmax'))
    return model


def local_CNN(input_shape):

    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())

    # flatten output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model



def local_CNN_LSTM(input_shape):

    # nb_filter = 50 # 50 output channels
    # kernel_size_x = input_shape[0]//10 # 50 -> 5
    # kernel_size_y = input_shape[1] # 48

    model = keras.Sequential()
    model.add(keras.layers.Conv2D(nb_filter, (kernel_size_x, kernel_size_y), input_shape=input_shape, activation='relu'))
    # model.add(MaxPool2D(pool_size=(2, 2))) by imposing kernel_size_y to #features
    model.add(keras.layers.Dropout(0.2))
    #     reshape(-1,data.shape[1]).shape
    model.add(keras.layers.Reshape(target_shape=(-1, nb_filter)))
    model.add(keras.layers.LSTM(64, return_sequences=False))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(2, activation='softmax'))
    print('Compile model...')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

def local_lstm_gru(input_shape):
    # build network topology
    model = keras.Sequential()

    # 1st conv layer
    model.add(keras.layers.LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.GRU(64))

    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    model.add(keras.layers.Dense(7, activation='softmax'))
    
    optimiser = keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimiser,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])

    return model


def local_GRU(input_shape):

    # build network topology
    model = keras.Sequential()

    model.add(keras.layers.GRU(128, input_shape=input_shape, return_sequences=True))
    model.add(keras.layers.GRU(64))

          # dense layer
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

          # output layer
    model.add(keras.layers.Dense(7, activation='softmax'))

    return model

In [11]:
comms_round = 100
loss='sparse_categorical_crossentropy'
metrics = ['accuracy']
optimiser = keras.optimizers.legacy.Adam(learning_rate=0.0001)   

In [12]:

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clients
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    #print(weight)
    #print(scalar)
    weight_final = []
    variances = []
    steps = len(weight)
    # variances.append(np.var(np.array(weight)))
    # inverse_variances = [1 / variance for variance in variances]
    # precision_weighted_average = np.average(weight[0], axis=0, weights=inverse_variances)
    # print(precision_weighted_average.shape)
    # print()
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final
    #return precision_weighted_average



def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        #print(layer_mean)
        #weight_ratios = tf.divide(grad_list_tuple, layer_mean)
        avg_grad.append(layer_mean)
        
    return avg_grad

 
def test_model(X_test, Y_test,  model, comm_round):
    
    #X_test = X_test[np.newaxis, ...]
    logits = model.predict(X_test)
    #logits = model.predict(X_test)
    #print(logits.shape)
    loss = tf.keras.losses.sparse_categorical_crossentropy(Y_test,logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), Y_test)
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

In [13]:
#initialize global model
input_shape=(X_train.shape[1], X_train.shape[2])
global_model = global_lstm_gru(input_shape)
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        local_model = local_lstm_gru(input_shape)
        local_model.compile(loss=loss, 
                      optimizer=optimiser, 
                      metrics=metrics)
        #local_model.summary()
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data'
        local_model.fit(clients_batched[client], epochs=1, verbose=2)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    #print(scaling_factor,scaled_local_weight_list)
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

70/70 - 14s - loss: 1.9604 - accuracy: 0.1509 - 14s/epoch - 205ms/step
70/70 - 13s - loss: 1.9307 - accuracy: 0.1888 - 13s/epoch - 190ms/step
35/35 [==============================] - 3s 43ms/step
comm_round: 0 | global_acc: 29.821% | global_loss: [2.0488088 1.880262  1.9265169 ... 1.7095804 1.9615457 1.941262 ]
70/70 - 16s - loss: 1.8241 - accuracy: 0.3067 - 16s/epoch - 229ms/step
70/70 - 14s - loss: 1.8119 - accuracy: 0.3232 - 14s/epoch - 202ms/step
35/35 [==============================] - 2s 45ms/step
comm_round: 1 | global_acc: 44.107% | global_loss: [1.9206913 1.6925352 1.7596598 ... 1.5259391 1.796106  1.9122516]
70/70 - 15s - loss: 1.6584 - accuracy: 0.4179 - 15s/epoch - 209ms/step
70/70 - 17s - loss: 1.6441 - accuracy: 0.4183 - 17s/epoch - 238ms/step
35/35 [==============================] - 2s 57ms/step
comm_round: 2 | global_acc: 50.536% | global_loss: [1.6054292 1.4111191 1.6351525 ... 1.2645589 1.3908286 1.4834512]
70/70 - 14s - loss: 1.4187 - accuracy: 0.5009 - 14s/epoch - 2

KeyboardInterrupt: 

In [ ]:
#initialize global model
input_shape=(X_train.shape[1], X_train.shape[2])
global_model = global_GRU(input_shape)
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        local_model = local_GRU(input_shape)
        local_model.compile(loss=loss, 
                      optimizer=optimiser, 
                      metrics=metrics)
        #local_model.summary()
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data'
        local_model.fit(clients_batched[client], epochs=1, verbose=2)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    #print(scaling_factor,scaled_local_weight_list)
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

70/70 - 12s - loss: 1.9321 - accuracy: 0.1946 - 12s/epoch - 177ms/step
70/70 - 13s - loss: 1.9107 - accuracy: 0.2067 - 13s/epoch - 191ms/step
35/35 [==============================] - 3s 47ms/step
comm_round: 0 | global_acc: 36.696% | global_loss: [1.6649901 1.768644  1.7759309 ... 1.472367  1.8892633 1.8147023]
